In [1]:
#importing all the things

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import ast
import seaborn as sns
sns.set()

%matplotlib inline

In [7]:
#initializing constants
START_DATE = '12/24/2016'
NUM_OF_WEEKS = 4

In [8]:
def get_file_names(start,number): #generates weeknums
    files = []
    ds = dt.datetime.strptime(start,"%m/%d/%Y")
    file_name = (ds.strftime("%y%m%d"))
    files.append(int(file_name))
    for i in range(1,number+1):
        ds = ds+ dt.timedelta(days=7)
        file_name = int(ds.strftime("%y%m%d"))
        files.append(file_name)
    print (files)
    return files


def get_data_parse_dt(week_nums): #imports files from mta website
    url = "turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)

def sortLinename(string):
    listLines = ''.join(sorted(string))
    return listLines

In [9]:
data = get_data_parse_dt(get_file_names(START_DATE,NUM_OF_WEEKS))
data.head()

[161224, 161231, 170107, 170114, 170121]


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,03:00:00,REGULAR,5967477,2022101
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,07:00:00,REGULAR,5967485,2022116
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,11:00:00,REGULAR,5967553,2022233
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,15:00:00,REGULAR,5967790,2022331
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/17/2016,19:00:00,REGULAR,5968186,2022421


In [ ]:
data.columns = [column.strip().lower() for column in turnstiles_df.columns] #formatting
data["date_time"] = pd.to_datetime(data.date + " " + data.time, format="%m/%d/%Y %H:%M:%S") #creating datetime column

In [ ]:
data.linename = data.linename.apply(sortLinename) #standardizing linenames

In [ ]:
(data
 .groupby(["c/a", "unit", "scp", "station", "date_time", "linename"])
 .entries.count()
 .reset_index()
 .sort_values("entries", ascending=False)).head(5)

In [ ]:
data = data.drop(["exits", "desc"], axis=1, errors="ignore")

In [ ]:
dailyData = (data.groupby(["c/a", "unit", "scp", "station", "date", "linename"],as_index=False)
                        .entries.first())

In [ ]:
dailyData[["prev_date", "prev_entries"]] = (dailyData.groupby(["c/a", "unit", "scp", "station", "linename"])
                                            ["date", "entries"].shift(1))
dailyData.dropna(subset=["prev_date"], axis=0, inplace=True)

In [ ]:
dailyData[dailyData["entries"] < dailyData["prev_entries"]]

In [ ]:
(dailyData[dailyData["entries"] < dailyData["prev_entries"]]
    .groupby(["c/a", "unit", "scp", "station"])
    .size())

In [ ]:
def get_daily_counts(row, max_counter):
    counter = abs(row["entries"] - row["prev_entries"])
    if counter > max_counter:
        print(row["entries"], row["prev_entries"])
        counter = min(row["entries"], row["prev_entries"])
    if counter > max_counter:
        return 0
    return counter

In [ ]:
dailyData["daily_entries"] = dailyData.apply(get_daily_counts, axis=1, max_counter=20000)